In [92]:
import pandas as pd
import nltk

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
# Importing libraries 
import nltk 
import re 
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer 
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
import pandas as pd 

In [93]:
file = 'F://Environmental Baseline Data//Version 4 - Final//Support files//Appendices mapped with files per Project.xlsx'

df = pd.read_excel(file, sheet_name="new")
df.head()
len(df)
#df.columns

Text = df['Component PDF Name']
print(len(Text))
Text[1]

,Application title short,Component,Component PDF Name,file_name,Component Index
0,Application for 2021 NGTL System Expansion Pro...,Appendix A,Environmental Protection Plan,A92619-16 16 ESA Appendix A EPP - A6F4Q9,1.0
1,Application for 2021 NGTL System Expansion Pro...,Appendix B,Environmental Alignment Sheet Package,A92619-17 17 ESA Appendix B EAS-Part A - A6F4R0,2.0
2,Application for 2021 NGTL System Expansion Pro...,Appendix B,Environmental Alignment Sheet Package,A92619-18 18 ESA Appendix B EAS-Part B - A6F4R1,3.0
3,Application for 2021 NGTL System Expansion Pro...,Appendix C,"Soil and Soil Productivity, Water Quality and ...",A92619-19 19 ESA Appendices C to K - A6F4R2,4.0
4,Application for 2021 NGTL System Expansion Pro...,ESA,ESA,A92619-10 10 ESA Sections 1 to 4 - A6F4Q3,5.0


1196

1196


'Environmental Alignment Sheet Package'

# Frequency, Bigrams, Trigrams

Ref -> https://www.geeksforgeeks.org/tf-idf-for-bigrams-trigrams/

In [145]:
#nltk.download('stopwords')
def remove_string_special_characters(s): 
      
    # removes special characters with ' ' 
    stripped = re.sub('[^a-zA-z\s]', '', s) 
    stripped = re.sub('_', '', stripped) 
      
    # Change any white space to one space 
    stripped = re.sub('\s+', ' ', stripped) 
      
    # Remove start and end white spaces 
    stripped = stripped.strip() 
    if stripped != '': 
            return stripped.lower() 
          

# Stopword removal  
stop_words = set(stopwords.words('english')) 
your_list = ['and', 'is', 'the', 'table', 'appendix', 'esa', 'environment', 'environmental', 
             'sheet', 'report', 'assessment', 'nan', 'plan', 'protection', 'section', 'effects',
            'project'] 
Text_clean = []
for line in Text:
    line_clean = ""
    line = remove_string_special_characters(str(line))
    for x in str(line).split():
        if (x not in stop_words) and (x not in your_list) and len(x) > 2:
            line_clean = line_clean + " " + str(x)
    Text_clean.append(line_clean.strip())
len(Text_clean)
Text_clean[5:13]


1196

['',
 '',
 '',
 '',
 '',
 'executive summary contents',
 'introduction',
 'introduction project description consultation relation routing siting']

# Frequency

In [146]:
Big_Text_clean = ""
for line in Text_clean:
    Big_Text_clean = Big_Text_clean + " " +line
    
#len(Big_Text_clean.split())

wordlist = Big_Text_clean.split()
wordfreq = []
for w in Big_Text_clean.split():
    wordfreq.append(wordlist.count(w))
    
len(wordlist)
len(wordfreq)

wordlist_unique = []
wordfreq_unique = []
for i in range(len(wordlist)):
    if wordlist[i] not in wordlist_unique:
        wordlist_unique.append(wordlist[i])
        wordfreq_unique.append(wordfreq[i])
        #print("{} : {}".format(wordlist[i], wordfreq[i]))
len(wordfreq_unique)
len(wordlist_unique)

df_wordlist = pd.DataFrame({'Word' : wordlist_unique,
                            'Freq' : wordfreq_unique})
df_wordlist.head()

df_wordlist_sort = df_wordlist.sort_values('Freq',ascending = False)
df_wordlist_sort.head(50)

3586

3586

364

364

,Word,Freq
0,alignment,446
1,package,33
2,soil,37
3,productivity,1
4,water,11


,Word,Freq
0,alignment,446
116,sheets,414
104,technical,206
287,fisheries,81
34,socioeconomic,61
54,vegetation,60
9,wildlife,57
79,gas,53
56,baseline,52
26,project,47


# Classification Based on these Keywords 

In [147]:
land1 = ['soil', 'land', 'ground', 'terrain', 'topography', 'ecozones']
air2 = ['air', 'emission', 'GHG', 'gas', 'greenhouse', 'weather' , 'climate', 'meteorological']
water3 = ['water', 'fish', 'wetlands', 'navigation', 'marine', 'aqua']
wildlife4 = ['wild', 'fish', 'species', 'habitat', 'acoustic']
vegetation5 = ['vegetation', 'wetlands', 'plant', 'soil']
human6 = ['human', 'socio', 'social', 'economic', 'economy' 'occupancy', 'heritage', 'health', 'aesthetics', 
          'employment', 'acoustic', 'traditional', 'navigation', 'resource', 'infrastructure', 'noise']
alignment_sheet7 = ['alignment', 'sheet']
tech8 = ['technical', 'tech']

In [161]:
def check_topic_present(topic, text):
    for key in topic:
        if key in text:
            return 1
    return 0

Text = df['Component PDF Name']
print(len(Text))
Text[1]
count_blanks = 0

topics = []
for line in Text:
    line = remove_string_special_characters(str(line)).lower()
    line_topics = []
    
    if check_topic_present(land1, line) == 1:
        line_topics.append('land')
        
    if check_topic_present(air2, line) == 1:
        line_topics.append('air')
    
    if check_topic_present(water3, line) == 1:
        line_topics.append('water')
        
    if check_topic_present(wildlife4, line) == 1:
        line_topics.append('wildlife')
        
    if check_topic_present(vegetation5, line) == 1:
        line_topics.append('vegetation')
        
    if check_topic_present(human6, line) == 1:
        line_topics.append('human')
        
    if check_topic_present(alignment_sheet7, line) == 1:
        line_topics.append('Alignment Sheet')
        
    if check_topic_present(tech8, line) == 1:
        line_topics.append('tech')
        
    if line_topics == []:
        count_blanks = count_blanks +1 
        
    topics.append(line_topics)

len(topics)
len(Text)
count_blanks

1196


'Environmental Alignment Sheet Package'

1196

1196

269

In [162]:
df['topics'] = topics
df.head()


,Application title short,Component,Component PDF Name,file_name,Component Index,topics
0,Application for 2021 NGTL System Expansion Pro...,Appendix A,Environmental Protection Plan,A92619-16 16 ESA Appendix A EPP - A6F4Q9,1.0,[]
1,Application for 2021 NGTL System Expansion Pro...,Appendix B,Environmental Alignment Sheet Package,A92619-17 17 ESA Appendix B EAS-Part A - A6F4R0,2.0,[Alignment Sheet]
2,Application for 2021 NGTL System Expansion Pro...,Appendix B,Environmental Alignment Sheet Package,A92619-18 18 ESA Appendix B EAS-Part B - A6F4R1,3.0,[Alignment Sheet]
3,Application for 2021 NGTL System Expansion Pro...,Appendix C,"Soil and Soil Productivity, Water Quality and ...",A92619-19 19 ESA Appendices C to K - A6F4R2,4.0,"[land, air, water, wildlife, vegetation, human]"
4,Application for 2021 NGTL System Expansion Pro...,ESA,ESA,A92619-10 10 ESA Sections 1 to 4 - A6F4Q3,5.0,[]


In [165]:
file_name = 'F://Environmental Baseline Data//Version 4 - Final//Support files//Table_Title_Topics_for_Brooke.csv'

df.to_csv(file_name, index = False, encoding='utf-8-sig')

In [ ]:
response = urllib.request.urlopen(url)
html = response.read()
text = obo.stripTags(html).lower()
fullwordlist = obo.stripNonAlphaNum(text)
wordlist = obo.removeStopwords(fullwordlist, obo.stopwords)
dictionary = obo.wordListToFreqDict(wordlist)
sorteddict = obo.sortFreqDict(dictionary)

for s in sorteddict: print(str(s))

In [ ]:
# html-to-freq-2.py
import urllib.request, urllib.error, urllib.parse
import obo



In [109]:
# Getting trigrams  
vectorizer = CountVectorizer(ngram_range = (3,3)) 
X1 = vectorizer.fit_transform(Text_clean)  
features = (vectorizer.get_feature_names()) 
print("\n\nFeatures : \n", features) 
print("\n\nX1 : \n", X1.toarray()) 
  
# Applying TFIDF 
vectorizer = TfidfVectorizer(ngram_range = (3,3)) 
X2 = vectorizer.fit_transform(Text_clean)
scores = (X2.toarray()) 
print("\n\nScores : \n", scores) 
  




Features : 
 ['abandonment plan traditional', 'aboriginal engagement activities', 'aboriginal engagement landowner', 'aboriginal engagement outcomes', 'aboriginal field study', 'accidents malfunctions section', 'accidents malfunctions unplanned', 'accounts techncial report', 'acid rock drainage', 'acoustic environment preliminary', 'acoustic environment section', 'acoustic environment technical', 'acoustic evaluation bakken', 'act clearance letters', 'activities considered cumulative', 'activities planned mitigation', 'activity watercource wildlife', 'additional undefined drainages', 'air emissions acoustic', 'air greenhouse gas', 'air quality assessment', 'air quality greenhouse', 'air quality modelling', 'air quality noise', 'air quality technical', 'aitken creek section', 'aitken kahta paragon', 'alberta british columbia', 'alberta technical report', 'alces river compressor', 'alignment sheet package', 'alignment sheets environmental', 'alignment sheets gas', 'alignment sheets gat

In [110]:
# Getting top ranking features 
sums = X2.sum(axis = 0) 
data1 = [] 
for col, term in enumerate(features): 
    data1.append( (term, sums[0,col] )) 
ranking = pd.DataFrame(data1, columns = ['term','rank']) 
words = (ranking.sort_values('rank', ascending = False)) 
print ("\n\nWords head : \n", words.head(7)) 



Words head : 
                                term        rank
256  environmental alignment sheets  392.205149
329      fisheries british columbia   25.980762
136       columbia technical report   25.980762
121      british columbia technical   25.980762
871     vegetation technical report   25.625859
30          alignment sheet package   23.334524
255   environmental alignment sheet   23.334524


# Bigrams

In [112]:
# Getting trigrams  
vectorizer = CountVectorizer(ngram_range = (2,2)) 
X1 = vectorizer.fit_transform(Text_clean)  
features = (vectorizer.get_feature_names()) 
print("\n\nFeatures : \n", features) 
print("\n\nX1 : \n", X1.toarray()) 
  
# Applying TFIDF 
vectorizer = TfidfVectorizer(ngram_range = (3,3)) 
X2 = vectorizer.fit_transform(Text_clean)
scores = (X2.toarray()) 
print("\n\nScores : \n", scores) 



Features : 
 ['abandonment assessment', 'abandonment plan', 'aboriginal engagement', 'aboriginal field', 'accidents malfunctions', 'accounts techncial', 'acid rock', 'acoustic assessments', 'acoustic environment', 'acoustic evaluation', 'act clearance', 'activities considered', 'activities planned', 'activity watercource', 'additional undefined', 'air emissions', 'air greenhouse', 'air quality', 'aitken creek', 'aitken kahta', 'alberta british', 'alberta technical', 'alces river', 'alignment sheet', 'alignment sheets', 'along liege', 'along northwest', 'alternative crossing', 'appendix air', 'appendix alternative', 'appendix assessment', 'appendix baseline', 'appendix conceptual', 'appendix field', 'appendix hydrology', 'appendix mitigation', 'appendix pathways', 'appendix predicted', 'appendix project', 'appendix socioeconomic', 'appendix soil', 'appendix watercourse', 'aquatic assessment', 'aquatic resources', 'aquatic technical', 'aquatics summary', 'aquatics technical', 'archaeol

In [113]:
# Getting top ranking features 
sums = X2.sum(axis = 0) 
data1 = [] 
for col, term in enumerate(features): 
    data1.append( (term, sums[0,col] )) 
ranking = pd.DataFrame(data1, columns = ['term','rank']) 
words = (ranking.sort_values('rank', ascending = False)) 
print ("\n\nWords head : \n", words.head(7)) 



Words head : 
                        term        rank
256  feasibility assessment  392.205149
329   inspection monitoring   25.980762
136     crossing assessment   25.980762
121   considered cumulative   25.980762
255           fall wildlife   23.334524
30      appendix assessment   23.334524
28             appendix air   21.920310


# Frequency 